In [42]:
import numpy as np
import heapq

In [43]:

class Graph:
  def __init__(self, numVertices):
    self.numVertices = numVertices
    self.adjMatrix = np.full((numVertices, numVertices), np.inf)
    self.minPaths = np.full((numVertices, 2), np.inf)

  def addEdge(self, src, dest, weight):
    self.adjMatrix[src][dest] = weight
    self.adjMatrix[dest][src] = weight

  def getWeight(self, src, dest):
    return self.adjMatrix[src][dest]
  
  def getNumVertices(self):
    return self.numVertices
  
  def printGraph(self):
    print(self.adjMatrix)

  def getNeighbors(self, vertex):
    indices = np.where(self.adjMatrix[vertex] != np.inf)
    neighbors = [(i, self.adjMatrix[vertex][i]) for i in indices[0]]
    return neighbors
  
  def computeMinPaths(self):
    for i in range(self.numVertices):
      sortedIndices = np.argsort(self.adjMatrix[i])
      self.minPaths[i] = (self.adjMatrix[i][sortedIndices[0]], self.adjMatrix[i][sortedIndices[1]])

class Node:
  def __init__(self, graph, prevNode=None, finalVertice=None):
    if prevNode is None or finalVertice is None:
      self.level = 0
      self.path = []
      self.cost = 0
      self.bound = bound(graph, self.path, self.cost, -1)
      return

    self.level = prevNode.level + 1
    self.path = np.concatenate([prevNode.path, [finalVertice]]).astype(int)
    if len(prevNode.path) == 0:
      self.cost = 0
    else:
      self.cost = prevNode.cost + graph.getWeight(prevNode.path[-1], finalVertice)
    self.bound = bound(graph, self.path, self.cost, finalVertice)

  def __lt__(self, other):
    return self.bound < other.bound
  
  def __gt__(self, other):
    return self.bound > other.bound
  
  def __eq__(self, other):
    return self.bound == other.bound
  
  def __str__(self) -> str:
    return f"Path: {self.path} Cost: {self.cost} Bound: {self.bound}"


def bound(graph, path, cost, finalVertice):
  unvisitedContributions = np.sum(graph.minPaths[np.isin(np.arange(graph.getNumVertices()), path) == False, :], axis=1)
  if finalVertice != -1:
    unvisitedContributions += graph.minPaths[finalVertice][0]
  return (cost + np.sum(unvisitedContributions)) / 2

def branchBoundTravelingSalesman(graph):
    graph.computeMinPaths()

    root = Node(graph, Node(graph), 0)
    queue = [root]
    heapq.heapify(queue)
    best = np.inf
    solution = []

    numVertices = graph.getNumVertices()

    while queue:
      node = heapq.heappop(queue)
      if node.level > numVertices:
        if node.cost < best:
          best = node.cost
          solution = node.path
      
      elif node.bound < best:
        if node.level < numVertices:
          for k in range(0, numVertices):
            if k not in node.path and graph.getWeight(node.path[-1], k) != np.inf and bound(graph, node.path, node.cost, k) < best:
              heapq.heappush(queue, Node(graph, node, k))

        elif graph.getWeight(node.path[-1], 0) != np.inf and bound(graph, node.path, node.cost, 0) < best:
          heapq.heappush(queue, Node(graph, node, 0))


    return solution, best

In [44]:
g = Graph(5)
g.addEdge(0, 1, 3)
g.addEdge(0, 2, 1)
g.addEdge(0, 3, 5)
g.addEdge(0, 4, 8)
g.addEdge(1, 2, 6)
g.addEdge(1, 3, 7)
g.addEdge(1, 4, 9)
g.addEdge(2, 3, 4)
g.addEdge(2, 4, 2)
g.addEdge(3, 4, 3)

# g.printGraph()
# print(g.getNeighbors(0))
# print(g.getNeighbors(1))
# print(g.getNeighbors(2))
# print(g.getNeighbors(3))
# print(g.getNeighbors(4))

g.computeMinPaths()
print(g.minPaths)

print('solution', branchBoundTravelingSalesman(g))


[[1. 3.]
 [3. 6.]
 [1. 2.]
 [3. 4.]
 [2. 3.]]
solution (array([0, 2, 4, 3, 1, 0]), 16.0)


In [45]:
g1 = Graph(4)
g1.adjMatrix = np.array([[0, 10, 15, 20],
       [10, 0, 35, 25],
       [15, 35, 0, 30],
       [20, 25, 30, 0]])

g1.printGraph()
g1.computeMinPaths()
print(g1.minPaths)

print('solution', branchBoundTravelingSalesman(g1))

[[ 0 10 15 20]
 [10  0 35 25]
 [15 35  0 30]
 [20 25 30  0]]
[[ 0. 10.]
 [ 0. 10.]
 [ 0. 15.]
 [ 0. 20.]]
solution (array([0, 2, 3, 1, 0]), 80)


In [46]:
root = Node(g, [], 0, 0)
queue = [root]
heapq.heapify(queue)
heapq.heappush(queue, Node(g, root.path, root.cost, 1))

for el in queue:
  print(el)


TypeError: __init__() takes from 2 to 4 positional arguments but 5 were given

In [ ]:
import heapq

heap = [3, 1, 4, 1, 5, 9, 2, 6, 5, 3, 5]
heapq.heapify(heap)
heapq.heappush(heap, 0)
print(heap)


[0, 1, 1, 3, 3, 2, 4, 6, 5, 5, 5, 9]
